In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df=pd.read_csv(r"C:\Users\Admin\Documents\SEM 5 CLG\AI\naukri_com-job_sample.csv")

In [6]:
df=df.drop(columns=['education','experience','industry','jobdescription','Unnamed: 5','numberofpositions','postdate','site_name','uniq_id'])

In [7]:
df.head(10)

,company,joblocation_address,jobtitle,payrate,skills
0,MM Media Pvt Ltd,Chennai,Walkin Data Entry Operator (night Shift),"1,50,000 - 2,25,000 P.A",ITES
1,find live infotech,Chennai,Work Based Onhome Based Part Time.,"1,50,000 - 2,50,000 P.A. 20000",Marketing
2,Softtech Career Infosystem Pvt. Ltd,Bengaluru,Pl/sql Developer - SQL,Not Disclosed by Recruiter,IT Software - Application Programming
3,Onboard HRServices LLP,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,Not Disclosed by Recruiter,Accounts
4,Spire Technologies and Solutions Pvt. Ltd.,Bengaluru,JAVA Technical Lead (6-8 yrs) -,Not Disclosed by Recruiter,IT Software - Application Programming
5,PFS Web Global Services Pvt Ltd,Bengaluru,WALK IN - As400 Developer - Pfsweb Global Serv...,Not Disclosed by Recruiter,IT Software - Application Programming
6,Kinesis Management Consultant Pvt. Ltd,"Delhi NCR, Mumbai, Bengaluru, Kochi, Greater N...",PHP Developer,"3,00,000 - 6,50,000 P.A",IT Software - Application Programming
7,Agile HR consultancy Pvt. Ltd. hiring for Ross...,Bengaluru,Member Technical Staff-wire Harness/cable Harn...,Not Disclosed by Recruiter,Production
8,HANSUM INDIA ELECTRONICS PVT.LTD.,Bengaluru,Team Leader,"1,00,000 - 2,00,000 P.A",Production
9,Accenture,Bengaluru,German Translator,Not Disclosed by Recruiter,ITES


In [8]:
df['jobtitle']=df['jobtitle'].str.strip()
df['jobtitle']=df['jobtitle'].str.lower()
df['skills']=df['skills'].str.strip()
df['skills']=df['skills'].str.lower()


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv=TfidfVectorizer(min_df=3, max_features=None,
                   strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}\S',
                   ngram_range=(1,3),
                   stop_words='english')
df['jobtitle']=df['jobtitle'].fillna('')
df['skills']=df['skills'].fillna('')
df['Job']=df['jobtitle']+df['skills']

In [10]:
tfv_matrix=tfv.fit_transform(df['Job'])

In [11]:
tfv_matrix

<22000x10711 sparse matrix of type '<class 'numpy.float64'>'
	with 183089 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [13]:
sig[0]

array([0.76163336, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [14]:
ind=pd.Series(df['jobtitle']).drop_duplicates(keep='first')
print(ind.index)

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            21988, 21990, 21991, 21992, 21993, 21994, 21995, 21997, 21998,
            21999],
           dtype='int64', length=17181)


In [15]:
index_=ind
indices=pd.Series(ind.index)
indices.index=index_

In [16]:
indices

jobtitle
walkin data entry operator (night shift)                                      0
work based onhome based part time.                                            1
pl/sql developer - sql                                                        2
manager/ad/partner - indirect tax - ca                                        3
java technical lead (6-8 yrs) -                                               4
walk in - as400 developer - pfsweb global services (bangalore)                5
php developer                                                                 6
member technical staff-wire harness/cable harness/technicians                 7
team leader                                                                   8
german translator                                                             9
business development manager                                                 10
opening for android developer-bangalore-4-8 yrs                              11
full stack web application (php

In [17]:
def give_rec(title , sig=sig):
    try:
        idx=indices[title]
        sig_scores=list(enumerate(sig[idx]))
        sig_scores=sorted(sig_scores, key=lambda x: x[1],reverse=True)
        sig_scores=sig_scores[1:11]
        customer_indices =[i[0] for i in sig_scores]
        return df.iloc[customer_indices]
        #values.sort_values(by = 'Experience' ,ascending=False)
       # return df.iloc[customer_indices]
        #return values.sort_values(by = 'Experience' ,ascending=False)
    except:
        contain_values=df[df['Job'].str.contains(title)]
        id=contain_values.index
        df.loc[id]
        if len(id)<15 :
            for j in range(0,len(id)): 
                if j==0:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[1:3]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores+sig_scores2[1:3]
        else :
             for j in range(5,15): 
                if j==5:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[1:3]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores+sig_scores2[1:3]
        customer_indices=[i[0] for i in sig_scores]
        print(df.iloc[customer_indices])
       # print(value.sort_values(by = 'Experience' ,ascending=False))

In [18]:
give_rec('developer')

,company,joblocation_address,jobtitle,payrate,skills,Job
1217,Ecentric Solutions Pvt. Ltd.,Bengaluru,jfs contractor developer,Not Disclosed by Recruiter,it software - application programming,jfs contractor developerit software - applicat...
1363,Confidential hiring for CMM Level 5,Bengaluru,mainframes developer,Not Disclosed by Recruiter,it software - application programming,mainframes developerit software - application ...
1385,Confidential hiring for CMM Level 5,Bengaluru,mssql developer,Not Disclosed by Recruiter,it software - application programming,mssql developerit software - application progr...
1388,Ernst Young,Bengaluru,blockchain developer,Not Disclosed by Recruiter,it software - application programming,blockchain developerit software - application ...
1431,Bob Technologies,Bengaluru,filenet/ datacap developer,Not Disclosed by Recruiter,it software - application programming,filenet/ datacap developerit software - applic...
2042,Millennium Technosoft Ltd.,Hyderabad / Secunderabad,qlickview developer,Not Disclosed by Recruiter,it software - application programming,qlickview developerit software - application p...
2365,Unitforce technologies Pvt. Ltd.,"Bengaluru/Bangalore , Hyderabad / Secunderabad",datapower developer,Not Disclosed by Recruiter,it software - application programming,datapower developerit software - application p...
2909,RATEIN INFOTECH (I) PVT. LTD.,Hyderabad / Secunderabad,developer,Not Disclosed by Recruiter,it software - application programming,developerit software - application programming
3292,CSC India Pvt Ltd.,"Bengaluru/Bangalore , Chennai , Mumbai , Noida...",obia developer,Not Disclosed by Recruiter,it software - application programming,obia developerit software - application progra...
5295,NetZealous Services India Pvt Ltd,"Bengaluru/Bangalore , bangalore",developer,Not Disclosed by Recruiter,it software - application programming,developerit software - application programming


In [19]:
give_rec('media planning')

                     company joblocation_address                   jobtitle  \
21912    Wipro Consumer Care           Bengaluru   manager - media planning   
14598  Bharti Airtel Limited               Noida            market planning   
9258                   MIRUM                 NaN  executive- media planning   
14598  Bharti Airtel Limited               Noida            market planning   

                          payrate     skills  \
21912  Not Disclosed by Recruiter  marketing   
14598  Not Disclosed by Recruiter  marketing   
9258   Not Disclosed by Recruiter  marketing   
14598  Not Disclosed by Recruiter  marketing   

                                      Job  
21912   manager - media planningmarketing  
14598            market planningmarketing  
9258   executive- media planningmarketing  
14598            market planningmarketing  


In [20]:
give_rec('technical support')

,company,joblocation_address,jobtitle,payrate,skills,Job
6214,INTROLLIGENT SOLUTIONS PVT LTD.,"Bengaluru/Bangalore , Bengaluru / Bangalore",technical support,Not Disclosed by Recruiter,ites,technical supportites
11595,AT&T INTELLECTUAL PROPERTY,Bengaluru/Bangalore,technical support,Not Disclosed by Recruiter,ites,technical supportites
11908,AT&T INTELLECTUAL PROPERTY,Bengaluru/Bangalore,technical support,Not Disclosed by Recruiter,ites,technical supportites
19525,Brainwork Technologies,Delhi,technical support,Not Disclosed by Recruiter,ites,technical supportites
862,Black and White Business Solutions Pvt Ltd hir...,Bengaluru,customer support or technical support,"2,00,000 - 4,00,000 P.A. Best in the industry ...",ites,customer support or technical supportites
308,Acelinkers Consultancy Services Pvt Ltd,"Bengaluru, Delhi, Delhi NCR, Gurgaon, Noida",team lead - technical support,"4,00,000 - 7,00,000 P.A",ites,team lead - technical supportites
10985,Crown Solutions India (P) Ltd,Bengaluru/Bangalore,excellent openings for freshers as a customer ...,Not Disclosed by Recruiter,ites,excellent openings for freshers as a customer ...
18070,Best RPO,"Bengaluru/Bangalore , Chennai , Noida",it analyst / it helpdesk / service desk / tech...,Not Disclosed by Recruiter,ites,it analyst / it helpdesk / service desk / tech...
21401,Mani Job Destination hiring for Leading Health...,"Hyderabad, Chennai",freshers - ar trainee (us) voice / customer ca...,"2,00,000 - 3,00,000 P.A",ites,freshers - ar trainee (us) voice / customer ca...
1451,The Calibre Quotient Consulting,Bengaluru,international customer support and internation...,"1,25,000 - 3,50,000 P.A. Plus incentives and 2...",ites,international customer support and internation...


In [21]:
give_rec('online bidding')

                             company             joblocation_address  \
10439  Cyberframe Infotech Pvt. Ltd.  Bengaluru/Bangalore , Ludhiana   
18811                       iHackers                           Delhi   

              jobtitle                     payrate skills                 Job  
10439    online bidder  Not Disclosed by Recruiter  sales  online biddersales  
18811  online marketer  Not Disclosed by Recruiter            online marketer  


In [22]:
give_rec('product engineer')

,company,joblocation_address,jobtitle,payrate,skills,Job
5797,Votary Softech Solutions Pvt. Ltd.,"Hyderabad / Secunderabad , Bengaluru/Bangalore",product engineer,Not Disclosed by Recruiter,it software - application programming,product engineerit software - application prog...
6654,DeltaX,"Bengaluru/Bangalore , Bengaluru / Bangalore",product engineer,Not Disclosed by Recruiter,it software - application programming,product engineerit software - application prog...
11585,Ugam Solutions Pvt Ltd,Bengaluru/Bangalore,product engineer,Not Disclosed by Recruiter,it software - application programming,product engineerit software - application prog...
157,Bob Technologies,Bengaluru,lead product engineer,Not Disclosed by Recruiter,it software - application programming,lead product engineerit software - application...
11630,Ugam Solutions Pvt Ltd,Bengaluru/Bangalore,sr. product engineer,Not Disclosed by Recruiter,it software - application programming,sr. product engineerit software - application ...
2003,Embinsys Technologies,Hyderabad / Secunderabad,core bsp product engineer,Not Disclosed by Recruiter,it software - application programming,core bsp product engineerit software - applica...
5884,Diginoc,Bengaluru/Bangalore,snr. product engineer,Not Disclosed by Recruiter,it software - application programming,snr. product engineerit software - application...
6390,infibeam,"Ahmedabad , Bengaluru/Bangalore",engineer,Not Disclosed by Recruiter,it software - application programming,engineerit software - application programming
6510,Power Research and Development Consultants Pvt...,"Bengaluru/Bangalore , Bengaluru / Bangalore",engineer,Not Disclosed by Recruiter,it software - application programming,engineerit software - application programming
6924,Dexterity Technologies,"Bengaluru/Bangalore , Chennai , Delhi/NCR , Hy...",system engineer,Not Disclosed by Recruiter,it software - application programming,system engineerit software - application progr...


In [23]:
give_rec('admin')

,company,joblocation_address,jobtitle,payrate,skills,Job
13762,Seven Consultancy,"Navi Mumbai, Thane(Turbhe)",admin,Not Disclosed by Recruiter,hr,adminhr
2901,Sandoz Pvt. Ltd.,Hyderabad / Secunderabad,payroll admin,Not Disclosed by Recruiter,hr,payroll adminhr
3157,Novartis Healthcare Pvt. Ltd.,Hyderabad / Secunderabad,payroll admin,Not Disclosed by Recruiter,hr,payroll adminhr
5225,Doowon Automotive India PVT LTD hiring for Doo...,Chennai,dgm - hr & admin,"7,50,000 - 8,00,000 P.A. Best in the industry",hr,dgm - hr & adminhr
13228,Ugam Solutions,Mumbai,sr. associate- admin,Not Disclosed by Recruiter,hr,sr. associate- adminhr
21328,Acculogix Software Solutions India (P) Ltd. hi...,Hyderabad,facility manager / facility admin,Not Disclosed by Recruiter,hr,facility manager / facility adminhr
0,MM Media Pvt Ltd,Chennai,walkin data entry operator (night shift),"1,50,000 - 2,25,000 P.A",ites,walkin data entry operator (night shift)ites
1,find live infotech,Chennai,work based onhome based part time.,"1,50,000 - 2,50,000 P.A. 20000",marketing,work based onhome based part time.marketing
2,Softtech Career Infosystem Pvt. Ltd,Bengaluru,pl/sql developer - sql,Not Disclosed by Recruiter,it software - application programming,pl/sql developer - sqlit software - applicatio...
3,Onboard HRServices LLP,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",manager/ad/partner - indirect tax - ca,Not Disclosed by Recruiter,accounts,manager/ad/partner - indirect tax - caaccounts


In [27]:
give_rec('manager')

,company,joblocation_address,jobtitle,payrate,skills,Job
3154,vCentric Technologies Pvt. Ltd.,Hyderabad / Secunderabad,display manager,Not Disclosed by Recruiter,it software - application programming,display managerit software - application progr...
3305,Arija software,Hyderabad / Secunderabad,manager,Not Disclosed by Recruiter,it software - application programming,managerit software - application programming
6442,Directing IT Systems Pvt. Ltd.,Bengaluru/Bangalore,it manager,Not Disclosed by Recruiter,it software - application programming,it managerit software - application programming
7344,HCL Technologies,Noida,manager,Not Disclosed by Recruiter,it software - application programming,managerit software - application programming
11251,Cognizant Technology Solutions India Ltd,"Chennai , Bengaluru/Bangalore",manager,Not Disclosed by Recruiter,it software - application programming,managerit software - application programming
13695,Tata Communications Ltd,Mumbai,manager,Not Disclosed by Recruiter,it software - application programming,managerit software - application programming
16493,URS Certification Ltd.,"Noida , Mumbai , Delhi",manager,Not Disclosed by Recruiter,it software - application programming,managerit software - application programming
19339,GeoShot Technologies,Delhi,manager,Not Disclosed by Recruiter,it software - application programming,managerit software - application programming
9660,Planman HR Private Limited,Bengaluru/Bangalore,engineer manager,Not Disclosed by Recruiter,it software - application programming,engineer managerit software - application prog...
14953,SKD GLOBAL,Delhi,sr. manager,Not Disclosed by Recruiter,it software - application programming,sr. managerit software - application programming
